In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()


In [ ]:
import shutil
from pathlib import Path
import os
from google.colab import drive
drive.mount('/content/drive')
# ---------- SOURCE ----------
SRC = Path("/content/drive/MyDrive/datasplit/train_val")

# ---------- DESTINATIONS ----------
STAGE1 = Path("/content/dataset_stage1")
STAGE2 = Path("/content/dataset_stage2")
STAGE3_STRUCT = Path("/content/dataset_stage3_structural")
STAGE3_HUMAN = Path("/content/dataset_stage3_human")

# ---------- CLASS GROUPS ----------
VEGETATION = ["Tree"]
MANMADE = ["Building", "Car", "Lab", "Person"]

STRUCTURAL = ["Building", "Lab"]
HUMAN = ["Person", "Car"]

# ---------- HELPER ----------
def copy_class(src_root, class_name, dst_root, dst_class):
    src = src_root / class_name
    dst = dst_root / dst_class
    dst.mkdir(parents=True, exist_ok=True)

    for img in src.iterdir():
        if img.is_file():
            shutil.copy(img, dst / img.name)

# ============================
# STAGE 1
# ============================
for folder in SRC.iterdir():
    if folder.name in VEGETATION:
        copy_class(SRC, folder.name, STAGE1, "Vegetation")
    elif folder.name in MANMADE:
        copy_class(SRC, folder.name, STAGE1, "ManMade")
    else:
        print(f"Skipping unexpected folder: {folder.name}")


# ============================
# STAGE 2
# ============================
for cls in STRUCTURAL:
    copy_class(SRC, cls, STAGE2, "Structural")

for cls in HUMAN:
    copy_class(SRC, cls, STAGE2, "Human")

# ============================
# STAGE 3
# ============================
for cls in STRUCTURAL:
    copy_class(SRC, cls, STAGE3_STRUCT, cls)

for cls in HUMAN:
    copy_class(SRC, cls, STAGE3_HUMAN, cls)

print("✅ All stage datasets created correctly.")


Mounted at /content/drive
✅ All stage datasets created correctly.


In [ ]:
import shutil
from pathlib import Path

# Source test directory
SRC_TEST = Path("/content/drive/MyDrive/datasplit/test")

# Destination test directory
DST_TEST = Path("/content/dataset_test")

# Create destination root
DST_TEST.mkdir(parents=True, exist_ok=True)

# Copy each class folder
for class_dir in SRC_TEST.iterdir():
    if not class_dir.is_dir():
        continue

    dst_class_dir = DST_TEST / class_dir.name
    dst_class_dir.mkdir(parents=True, exist_ok=True)

    for img in class_dir.iterdir():
        if img.is_file():
            shutil.copy(img, dst_class_dir / img.name)

print("✅ dataset_test copied correctly (unchanged 5-class structure).")


✅ dataset_test copied correctly (unchanged 5-class structure).


In [ ]:
from PIL import Image
import os

def clean_corrupted_images(root_dir):
    bad_files = []

    for root, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # verify image integrity
            except Exception:
                bad_files.append(file_path)

    for file_path in bad_files:
        os.remove(file_path)
        print(f"❌ Removed corrupted file: {file_path}")

    print(f"\n✅ Cleaning complete. Removed {len(bad_files)} files.")
clean_corrupted_images("/content/dataset_stage1")
clean_corrupted_images("/content/dataset_stage2")
clean_corrupted_images("/content/dataset_stage3_human")
clean_corrupted_images("/content/dataset_stage3_structural")
clean_corrupted_images("/content/dataset_test")


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


❌ Removed corrupted file: /content/dataset_stage1/ManMade/Car2_20251109_142533.jpg

✅ Cleaning complete. Removed 1 files.
❌ Removed corrupted file: /content/dataset_stage2/Human/Car2_20251109_142533.jpg

✅ Cleaning complete. Removed 1 files.
❌ Removed corrupted file: /content/dataset_stage3_human/Car/Car2_20251109_142533.jpg

✅ Cleaning complete. Removed 1 files.

✅ Cleaning complete. Removed 0 files.

✅ Cleaning complete. Removed 0 files.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
IMG_SIZE = (224, 224)
BATCH = 32

train_gen_s1 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    zoom_range=0.3,
    height_shift_range=0.05,
    width_shift_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_gen_s1 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_s1 = train_gen_s1.flow_from_directory(
    "/content/dataset_stage1",
    subset="training",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)

val_s1 = val_gen_s1.flow_from_directory(
    "/content/dataset_stage1",
    subset="validation",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)


Found 2280 images belonging to 2 classes.
Found 455 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False

model_s1 = Sequential([
    base,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_s1.compile(
    optimizer=Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model_s1.fit(
    train_s1,
    validation_data=val_s1,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 492s 7s/step - accuracy: 0.7445 - loss: 0.5559 - val_accuracy: 0.8813 - val_loss: 0.2760
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 467s 6s/step - accuracy: 0.9348 - loss: 0.1804 - val_accuracy: 0.9363 - val_loss: 0.1818
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 463s 6s/step - accuracy: 0.9369 - loss: 0.1791 - val_accuracy: 0.9451 - val_loss: 0.1373
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 441s 6s/step - accuracy: 0.9462 - loss: 0.1278 - val_accuracy: 0.9560 - val_loss: 0.1052
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 463s 6s/step - accuracy: 0.9481 - loss: 0.1269 - val_accuracy: 0.9604 - val_loss: 0.0965
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 454s 6s/step - accuracy: 0.9590 - loss: 0.1120 - val_accuracy: 0.9648 - val_loss: 0.0948
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 455s 6s/step - accuracy: 0.9583 - loss: 0.1080 - val_accuracy: 0.9692 - val_loss: 0.0833
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 494s 6s/step - accuracy: 0.9677 - loss: 0.0854 - val_accuracy: 0.9758 - v

In [ ]:
stage1_class_names = ["ManMade", "Vegetation"]


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = val_s1.classes
y_pred = (model_s1.predict(val_s1) > 0.36).astype(int).ravel()

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=val_s1.class_indices.keys()))

15/15 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step
[[299   4]
 [  8 144]]
              precision    recall  f1-score   support

     ManMade       0.97      0.99      0.98       303
  Vegetation       0.97      0.95      0.96       152

    accuracy                           0.97       455
   macro avg       0.97      0.97      0.97       455
weighted avg       0.97      0.97      0.97       455



In [ ]:
model_s1.save("/content/drive/MyDrive/model_stage1.h5")

NameError: name 'model_s1' is not defined

In [ ]:
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.99      0.98       303
           1       0.97      0.95      0.96       152

    accuracy                           0.97       455
   macro avg       0.97      0.97      0.97       455
weighted avg       0.97      0.97      0.97       455



In [ ]:
IMG_SIZE = (224, 224)
BATCH = 32

train_gen_s2 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_gen_s2 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_s2 = train_gen_s2.flow_from_directory(
    "/content/dataset_stage2",
    subset="training",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)

val_s2 = val_gen_s2.flow_from_directory(
    "/content/dataset_stage2",
    subset="validation",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

Found 1216 images belonging to 2 classes.
Found 302 images belonging to 2 classes.


In [ ]:
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False

model_s2 = Sequential([
    base,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_s2.compile(
    optimizer=Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model_s2.fit(
    train_s2,
    validation_data=val_s2,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 324s 8s/step - accuracy: 0.6369 - loss: 0.7613 - val_accuracy: 0.8841 - val_loss: 0.3844
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 284s 7s/step - accuracy: 0.9039 - loss: 0.2463 - val_accuracy: 0.9172 - val_loss: 0.2784
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 324s 8s/step - accuracy: 0.9342 - loss: 0.1754 - val_accuracy: 0.9238 - val_loss: 0.2326
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 277s 7s/step - accuracy: 0.9464 - loss: 0.1622 - val_accuracy: 0.9272 - val_loss: 0.1969
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 316s 8s/step - accuracy: 0.9496 - loss: 0.1243 - val_accuracy: 0.9536 - val_loss: 0.1644
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 284s 7s/step - accuracy: 0.9587 - loss: 0.1136 - val_accuracy: 0.9570 - val_loss: 0.1437
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 314s 8s/step - accuracy: 0.9525 - loss: 0.1204 - val_accuracy: 0.9669 - val_loss: 0.1271
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 283s 7s/step - accuracy: 0.9649 - loss: 0.1228 - val_accuracy: 0.9603 - v

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = val_s2.classes
y_pred = (model_s2.predict(val_s2) > 0.36).astype(int).ravel()

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=val_s2.class_indices.keys()))

10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 4s/step
[[158   4]
 [  7 133]]
              precision    recall  f1-score   support

       Human       0.96      0.98      0.97       162
  Structural       0.97      0.95      0.96       140

    accuracy                           0.96       302
   macro avg       0.96      0.96      0.96       302
weighted avg       0.96      0.96      0.96       302



In [ ]:
model_s2.save("/content/drive/MyDrive/model_stage2.h5")

In [ ]:
IMG_SIZE = (224, 224)
BATCH = 32

train_gen_s3 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_gen_s3 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_s3 = train_gen_s3.flow_from_directory(
    "/content/dataset_stage3_human",
    subset="training",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)

val_s3 = val_gen_s3.flow_from_directory(
    "/content/dataset_stage3_human",
    subset="validation",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

Found 652 images belonging to 2 classes.
Found 162 images belonging to 2 classes.


In [ ]:
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False

model_s3 = Sequential([
    base,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_s3.compile(
    optimizer=Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model_s3.fit(
    train_s3,
    validation_data=val_s3,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 165s 7s/step - accuracy: 0.6514 - loss: 0.7427 - val_accuracy: 0.9815 - val_loss: 0.2761
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 140s 7s/step - accuracy: 0.9197 - loss: 0.1701 - val_accuracy: 0.9938 - val_loss: 0.1680
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 128s 6s/step - accuracy: 0.9456 - loss: 0.1051 - val_accuracy: 1.0000 - val_loss: 0.1283
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 139s 7s/step - accuracy: 0.9762 - loss: 0.0650 - val_accuracy: 0.9938 - val_loss: 0.1002
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 125s 6s/step - accuracy: 0.9834 - loss: 0.0480 - val_accuracy: 0.9938 - val_loss: 0.0811
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 128s 6s/step - accuracy: 0.9826 - loss: 0.0474 - val_accuracy: 0.9938 - val_loss: 0.0675
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.9700 - loss: 0.0676 - val_accuracy: 0.9938 - val_loss: 0.0562
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - accuracy: 0.9870 - loss: 0.0313 - val_accuracy: 1.0000 - v

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = val_s3.classes
y_pred = (model_s3.predict(val_s3) > 0.36).astype(int).ravel()

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=val_s3.class_indices.keys()))

6/6 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step
[[87  0]
 [ 0 75]]
              precision    recall  f1-score   support

         Car       1.00      1.00      1.00        87
      Person       1.00      1.00      1.00        75

    accuracy                           1.00       162
   macro avg       1.00      1.00      1.00       162
weighted avg       1.00      1.00      1.00       162



In [ ]:
model_s3.save("/content/drive/MyDrive/model_stage3_human.h5")

In [ ]:
IMG_SIZE = (224, 224)
BATCH = 32

train_gen_st3 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_gen_st3 = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_st3 = train_gen_st3.flow_from_directory(
    "/content/dataset_stage3_structural",
    subset="training",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)

val_st3 = val_gen_s3.flow_from_directory(
    "/content/dataset_stage3_structural",
    subset="validation",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    shuffle=False
)

Found 564 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [ ]:
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base.trainable = False

model_st3 = Sequential([
    base,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model_st3.compile(
    optimizer=Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model_st3.fit(
    train_st3,
    validation_data=val_st3,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 182s 9s/step - accuracy: 0.7131 - loss: 0.6623 - val_accuracy: 0.9929 - val_loss: 0.2453
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 151s 8s/step - accuracy: 0.9384 - loss: 0.1739 - val_accuracy: 0.9857 - val_loss: 0.1470
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 143s 8s/step - accuracy: 0.9617 - loss: 0.1025 - val_accuracy: 0.9929 - val_loss: 0.1025
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 143s 8s/step - accuracy: 0.9881 - loss: 0.0510 - val_accuracy: 1.0000 - val_loss: 0.0769
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 143s 8s/step - accuracy: 0.9856 - loss: 0.0402 - val_accuracy: 1.0000 - val_loss: 0.0559
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 153s 8s/step - accuracy: 0.9905 - loss: 0.0273 - val_accuracy: 1.0000 - val_loss: 0.0440
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 147s 8s/step - accuracy: 0.9997 - loss: 0.0174 - val_accuracy: 1.0000 - val_loss: 0.0352
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 146s 8s/step - accuracy: 0.9816 - loss: 0.0350 - val_accuracy: 1.0000 - v

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = val_st3.classes
y_pred = (model_st3.predict(val_st3) > 0.36).astype(int).ravel()

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=val_st3.class_indices.keys()))

5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 4s/step
[[63  1]
 [ 0 76]]
              precision    recall  f1-score   support

    Building       1.00      0.98      0.99        64
         Lab       0.99      1.00      0.99        76

    accuracy                           0.99       140
   macro avg       0.99      0.99      0.99       140
weighted avg       0.99      0.99      0.99       140



In [ ]:
model_st3.save("/content/drive/MyDrive/model_stage3_structural.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

TEST_DIR = "/content/dataset_test"

test_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
).flow_from_directory(
    TEST_DIR,
    target_size=(224,224),
    batch_size=1,
    class_mode="categorical",
    shuffle=False
)


Found 401 images belonging to 5 classes.


In [ ]:
class_names = list(test_gen.class_indices.keys())
print(class_names)


['Building', 'Car', 'Lab', 'Person', 'Tree']


In [ ]:
STAGE1_CLASSES = ['ManMade', 'Vegetation']
STAGE2_CLASSES = ['Human', 'Structural']
STAGE3_HUMAN_CLASSES = ['Car', 'Person']
STAGE3_STRUCT_CLASSES = ['Building', 'Lab']


In [ ]:
y_true, y_pred = [], []

for i in range(len(test_gen)):
    img, label = test_gen[i]
    true = class_names[np.argmax(label)]
    y_true.append(true)

    # ---------- STAGE 1 (Binary Sigmoid) ----------
    p1 = model_s1.predict(img, verbose=0)[0][0]
    if p1 < 0.37:   # or 0.37 if tuned
        y_pred.append("Tree")
        continue

    # ---------- STAGE 2 (Binary Sigmoid) ----------
    p2 = model_s2.predict(img, verbose=0)[0][0]
    if p2 > 0.5:
        # ---------- STAGE 3 HUMAN ----------
        p3 = model_s3.predict(img, verbose=0)[0][0]
        y_pred.append("Person" if p3 > 0.5 else "Car")
    else:
        # ---------- STAGE 3 STRUCTURAL ----------
        p3 = model_st3.predict(img, verbose=0)[0][0]
        y_pred.append("Lab" if p3 > 0.5 else "Building")


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


[[ 51   2   0   2   2]
 [  3  73   0   0   1]
 [  0   0  66   1   0]
 [  1   1   0  64   0]
 [  2   4   0   2 126]]
              precision    recall  f1-score   support

    Building       0.89      0.89      0.89        57
         Car       0.91      0.95      0.93        77
         Lab       1.00      0.99      0.99        67
      Person       0.93      0.97      0.95        66
        Tree       0.98      0.94      0.96       134

    accuracy                           0.95       401
   macro avg       0.94      0.95      0.94       401
weighted avg       0.95      0.95      0.95       401



In [ ]:
s1.summary()
s2.summary()
s3_h.summary()
s3_s.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,055,974 (15.47 MB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,055,974 (15.47 MB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:
from collections import Counter
print(Counter(y_pred))


Counter({'Tree': 129, 'Car': 80, 'Person': 69, 'Lab': 66, 'Building': 57})


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
print("Downloading and loading CIFAR-100 data...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

# 2. Define the target class index
# In CIFAR-100, 'bicycle' is index 8
BICYCLE_LABEL = 8

# 3. Filter the Training Data
# We look for indices where the label equals 8 and keep only those images
train_mask = np.where(y_train == BICYCLE_LABEL)[0]
x_train_bicycles = x_train[train_mask]
y_train_bicycles = y_train[train_mask]

# 4. Filter the Test Data
test_mask = np.where(y_test == BICYCLE_LABEL)[0]
x_test_bicycles = x_test[test_mask]
y_test_bicycles = y_test[test_mask]

# 5. Output results
print("\n--- Filtering Complete ---")
print(f"Bicycle Train Images: {x_train_bicycles.shape}") # Should be (500, 32, 32, 3)
print(f"Bicycle Test Images:  {x_test_bicycles.shape}")

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step

--- Filtering Complete ---
Bicycle Train Images: (500, 32, 32, 3)
Bicycle Test Images:  (100, 32, 32, 3)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. Load the dataset
print("Loading CIFAR-100 data...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

# 2. Define the index for 'bus'
# In CIFAR-100 alphabetical order, 'bus' is index 13
BUS_LABEL = 13

# 3. Filter Training Data
train_mask = np.where(y_train == BUS_LABEL)[0]
x_train_bus = x_train[train_mask]
y_train_bus = y_train[train_mask]

# 4. Filter Test Data
test_mask = np.where(y_test == BUS_LABEL)[0]
x_test_bus = x_test[test_mask]
y_test_bus = y_test[test_mask]

# 5. Output results
print("\n--- Filtering Complete ---")
print(f"Bus Train Images: {x_train_bus.shape}")
print(f"Bus Test Images:  {x_test_bus.shape}")




Loading CIFAR-100 data...

--- Filtering Complete ---
Bus Train Images: (500, 32, 32, 3)
Bus Test Images:  (100, 32, 32, 3)


In [ ]:
def preprocess_cifar(img):
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img


In [ ]:
s1_conf = []
s2_conf = []

for img in x_train_bus[:20]:  # sample
    img = preprocess_cifar(img)
    img = tf.expand_dims(img, 0)

    p1 = model_s1.predict(img)[0][0]  # ManMade prob
    p2 = model_s2.predict(img)[0][0]  # Human prob

    s1_conf.append(p1)
    s2_conf.append(p2)

print("Stage1 vegetation mean:", np.mean(s1_conf))
print("Stage2 structural mean:", np.mean(s2_conf))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
from pathlib import Path

BASE = Path("/content/dataset_stage3_human_expanded")

(CLASS_CAR := BASE / "Car").mkdir(parents=True, exist_ok=True)
(CLASS_PERSON := BASE / "Person").mkdir(parents=True, exist_ok=True)
(CLASS_BUS := BASE / "Bus").mkdir(parents=True, exist_ok=True)

BASE


PosixPath('/content/dataset_stage3_human_expanded')

In [ ]:
import shutil

OLD_HUMAN = Path("/content/dataset_stage3_human")

shutil.copytree(OLD_HUMAN / "Car", CLASS_CAR, dirs_exist_ok=True)
shutil.copytree(OLD_HUMAN / "Person", CLASS_PERSON, dirs_exist_ok=True)

print("✔ Copied Car & Person")


✔ Copied Car & Person


In [ ]:
from PIL import Image
import numpy as np

def save_cifar_images(images, target_dir):
    target_dir = Path(target_dir)
    for i, img in enumerate(images):
        img = Image.fromarray(img)
        img = img.resize((224, 224))   # match EfficientNet input
        img.save(target_dir / f"bus_{i}.jpg")

save_cifar_images(x_train_bus, CLASS_BUS)

print("✔ Saved Bus images:", len(os.listdir(CLASS_BUS)))


✔ Saved Bus images: 500


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH = 32

datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    BASE,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    BASE,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print(train_gen.class_indices)


Found 1052 images belonging to 3 classes.
Found 262 images belonging to 3 classes.
{'Bus': 0, 'Car': 1, 'Person': 2}


In [ ]:
from tensorflow.keras import layers, models, optimizers

base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)

base.trainable = False   # freeze backbone

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(train_gen.num_classes, activation="softmax")(x)

model_s3_h_new = models.Model(inputs=base.input, outputs=outputs)

model_s3_h_new.compile(
    optimizer=optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_s3_h_new.summary()


Model: "functional_64"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_8         │ (None, 224, 224,  │          0 │ input_layer_8[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 224, 224,  │          7 │ rescaling_8[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_9         │ (None, 224, 224,  │          0 │ normalization_4[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_9[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,378,278 (16.70 MB)

 Trainable params: 328,707 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early = EarlyStopping(
    patience=3,
    restore_best_weights=True,
    monitor="val_loss"
)

history = model_s3_h_new.fit(
    train_gen,
    validation_data=val_gen,
    epochs=12,
    callbacks=[early]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 177s 5s/step - accuracy: 0.8651 - loss: 0.3316 - val_accuracy: 0.9962 - val_loss: 0.0117
Epoch 2/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 186s 6s/step - accuracy: 1.0000 - loss: 0.0069 - val_accuracy: 0.9962 - val_loss: 0.0108
Epoch 3/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 187s 6s/step - accuracy: 0.9992 - loss: 0.0047 - val_accuracy: 0.9962 - val_loss: 0.0086
Epoch 4/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 157s 5s/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.9924 - val_loss: 0.0134
Epoch 5/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 154s 5s/step - accuracy: 0.9976 - loss: 0.0042 - val_accuracy: 0.9962 - val_loss: 0.0132
Epoch 6/12
33/33 ━━━━━━━━━━━━━━━━━━━━ 151s 5s/step - accuracy: 0.9999 - loss: 0.0014 - val_accuracy: 0.9885 - val_loss: 0.0325


In [ ]:
model_s3_h_new.save("/content/drive/MyDrive/stage3_human_expanded.h5")
print("✔ Saved new Stage-3 Human model")


✔ Saved new Stage-3 Human model


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

IMG_SIZE = (224,224)
BATCH = 32

# Load models
s1 = tf.keras.models.load_model("/content/drive/MyDrive/model_stage1.h5", compile=False)
s2 = tf.keras.models.load_model("/content/drive/MyDrive/model_stage2.h5", compile=False)
s3_h = tf.keras.models.load_model("/content/drive/MyDrive/stage3_human_expanded.h5", compile=False)   # NEW
s3_s = tf.keras.models.load_model("/content/drive/MyDrive/model_stage3_structural.h5", compile=False)

test_datagen_final = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_gen_final = test_datagen_final.flow_from_directory(
    "/content/drive/MyDrive/datasplit/test",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    shuffle=False
)

class_names = list(test_gen_final.class_indices.keys())
print("Test classes:", class_names)


Found 401 images belonging to 5 classes.
Test classes: ['Building', 'Car', 'Lab', 'Person', 'Tree']


In [ ]:
STAGE1_THRESHOLD = 0.37   # tuned earlier (Vegetation prob threshold)

def predict_hierarchical(batch):
    preds = []

    for img in batch:
        img = np.expand_dims(img, axis=0)

        # ---------- STAGE 1 ----------
        p1 = model_s1.predict(img, verbose=0)[0][0]     # P(Vegetation)

        if p1 > STAGE1_THRESHOLD:
            preds.append("Tree")
            continue

        # ---------- STAGE 2 ----------
        p2 = model_s2.predict(img, verbose=0)[0][0]     # P(Structural)

        if p2 > 0.5:
            # ---------- STAGE 3 STRUCTURAL ----------
            p3 = model_st3.predict(img, verbose=0)[0][0]
            y_pred.append("Lab" if p3 > 0.5 else "Building")
        else:
            # ---------- STAGE 3 HUMAN ----------
            p3 = model_s3_h_new.predict(img, verbose=0)[0]
            label = ["Bus","Car","Person"][np.argmax(p3)]

        preds.append(label)

    return preds


In [ ]:
y_true = []
y_pred = []

for i in range(len(test_gen_final)):
    imgs, labels = test_gen_final[i]

    preds = predict_hierarchical(imgs)

    # ground truth labels (original dataset)
    gt = [class_names[np.argmax(l)] for l in labels]

    y_true.extend(gt)
    y_pred.extend(preds)

    if len(y_true) >= test_gen_final.samples:
        break


NameError: name 'model_s1' is not defined

In [ ]:
print(confusion_matrix(y_true, y_pred))
print()
print(classification_report(y_true, y_pred, digits=3))


[[ 51   3   0   1   2]
 [  3  73   0   0   1]
 [ 66   0   0   1   0]
 [  1   1   0  64   0]
 [  2   4   0   2 126]]

              precision    recall  f1-score   support

    Building      0.415     0.895     0.567        57
         Car      0.901     0.948     0.924        77
         Lab      0.000     0.000     0.000        67
      Person      0.941     0.970     0.955        66
        Tree      0.977     0.940     0.958       134

    accuracy                          0.783       401
   macro avg      0.647     0.751     0.681       401
weighted avg      0.713     0.783     0.735       401



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
s1 = load_model(S1_PATH, compile=False)
s2 = load_model(S2_PATH, compile=False)
s3_h = load_model(S3_H_PATH, compile=False)
s3_s = load_model(S3_S_PATH, compile=False)


In [ ]:
from tensorflow.keras.models import load_model
s1 = load_model("/content/drive/MyDrive/model_stage1.h5", compile=False)
s2 = load_model("/content/drive/MyDrive/model_stage2.h5", compile=False)
s3_h = load_model("/content/drive/MyDrive/stage3_human_expanded.h5", compile=False)
s3_s = load_model("/content/drive/MyDrive/model_stage3_structural.h5", compile=False)
